## Linking in Spark



In [1]:
from splink.spark.jar_location import similarity_jar_location

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import types

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "16")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

23/05/02 15:41:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pandas as pd 
df = spark.read.csv("./data/fake_1000.csv", header=True)

In [3]:
import splink.spark.spark_comparison_library as cl
import splink.spark.spark_comparison_template_library as ctl

settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        ctl.name_comparison("first_name"),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("email"),
    ],
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "em_convergence": 0.01
}

In [4]:
from splink.spark.spark_linker import SparkLinker
linker = SparkLinker(df, settings)
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)


--WARN-- 
 You are using datediff comparison
                        with str-casting and ANSI is not enabled. Bad dates
                        e.g. 1999-13-54 will not trigger an exception but will
                        classed as comparison level = "ELSE". Ensure date strings
                        are cleaned to remove bad dates 

Probability two random records match is estimated to be  0.00389.               
This means that amongst all possible pairwise record comparisons, one in 257.25 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,941.67 matching pairs


In [5]:
linker.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
23/05/02 15:41:58 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink_demos/tmp_checkpoints/a403755a-803e-4687-b331-363181a178bc/__splink__df_concat_with_tf_461c6dc53
                                                                                
Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [6]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname
23/05/02 15:42:07 WARN DataSource: All paths were ignored:                      
  file:/Users/rosskennedy/splink_demos/tmp_checkpoints/a403755a-803e-4687-b331-363181a178bc/__splink__df_comparison_vectors_45d1e013c

Iteration 1: Largest change in params was -0.539 in the m_probability of dob, level `Exact match`
Iteration 2: Largest change in params was 0.0385 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.00854 in probability_two_random_records_match

EM converged after 3 iterations

Your model is not yet fully trained. Missing estimates for:
    - first_n

In [7]:
results = linker.predict(threshold_match_probability=0.9)

23/05/02 15:42:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/05/02 15:42:27 ERROR CodeGenerator: failed to compile: org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "bhj_doConsume_0$(Lorg/apache/spark/sql/catalyst/expressions/GeneratedClass$GeneratedIteratorForCodegenStage3;Lorg/apache/spark/unsafe/types/UTF8String;Lorg/apache/spark/unsafe/types/UTF8String;Lorg/apache/spark/unsafe/types/UTF8String;ZLorg/apache/spark/unsafe/types/UTF8String;ZLorg/apache/spark/unsafe/types/UTF8String;ZLorg/apache/spark/unsafe/types/UTF8String;ZDZ)V" of class "org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3" grows beyond 64 KB
org.codehaus.janino.InternalCompilerException: Compiling "GeneratedClass" in "generated.java": Code of method "bhj_doConsume_0$(Lorg/apache/spark/sql/catalyst/

In [8]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,...,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,12.418414,0.999817,63,67,Adam,Adam,3,86.236085,Jsneo,Jonse,...,-1,0.019680,NaN,1.000000,1.000000,ajones6@cortez-wilcox.com,ajones6@cortez-wilcox.com,3,256.78781,0
1,18.918572,0.999998,226,228,Julia,Julia,3,86.236085,Smith,Smith,...,1,0.003690,0.003690,10.269631,14.944992,julia.smith@english.org,julia.smith@english.org,3,256.78781,0
2,4.023966,0.942089,967,969,Joseph,Joseph,3,86.236085,Davsi,None,...,0,0.212792,0.007380,0.458966,1.000000,joseph.d@bell.com,joseph.d@bell.com,3,256.78781,0
3,20.090066,0.999999,305,308,Harley,Harley,3,86.236085,Rose,Ros,...,-1,NaN,0.012300,1.000000,1.000000,harley.rose50@sawyer-lin.org,harley.rose50@sawyer-lin.org,3,256.78781,0
4,13.117318,0.999887,657,658,Benjamin,Benjamin,3,86.236085,None,Nemena,...,0,0.002460,0.212792,0.458966,1.000000,benjamin.newman@johnson.com,benjamin.newman@johnson.com,3,256.78781,0
